<a href="https://colab.research.google.com/github/MhmdIlham10/Temu-Kembali-Informasi/blob/main/TKI_Jakarta_Barat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.5 MB/s eta 0:00:00


In [17]:
import argparse
import io
import os
import re
import time
import urllib
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import date

import pandas as pd
import requests
from bs4 import BeautifulSoup
from pdfminer import high_level

def create_path(folder_name):
    path = os.path.join(os.getcwd(), folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def open_page(link):
    count = 0
    while count < 3:
        try:
            return BeautifulSoup(requests.get(link).text, "html.parser")
        except:
            count += 1
            time.sleep(5)

def get_detail(soup, keywoard):
    try:
        text = (
            soup.find(lambda tag: tag.name == "td" and keywoard in tag.text)
            .find_next("td")
            .get_text()
            .strip()
        )
        return text
    except:
        return ""

def get_pdf(url, path_pdf):
    file = urllib.request.urlopen(url)
    file_name = file.info().get_filename().replace("/", " ")
    file_content = file.read()
    with open(f"{path_pdf}/{file_name}", "wb") as out_file:
        out_file.write(file_content)
    return io.BytesIO(file_content), file_name

def clean_text(text):
    text = text.replace("M a h ka m a h A g u n g R e p u blik In d o n esia\n", "")
    text = text.replace("Disclaimer\n", "")
    text = text.replace(
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencamtumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n",
        "",
    )
    text = text.replace(
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahn teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n",
        "",
    )
    text = text.replace(
        "Email: kepaniteraan@mahkamahagung.go.id  Telp: 021-384 3348 (ext.318)\n",
        "",
    )
    return text

def extract_data(link, keywoard_url):
    global today
    global path_output
    global path_pdf
    global download_pdf

    path_output = create_path("txt_putusan")
    path_pdf = create_path("pdf_putusan")
    today = date.today().strftime("%Y-%m-%d")

    soup = open_page(link)
    table = soup.find("table", {"class": "table"})
    judul = table.find("h2").text
    table.find("h2").decompose()

    nomor = get_detail(table, "Nomor")
    tingkat_proses = get_detail(table, "Tingkat Proses")
    klasifikasi = get_detail(table, "Klasifikasi")
    kata_kunci = get_detail(table, "Kata Kunci")
    tahun = get_detail(table, "Tahun")
    tanggal_register = get_detail(table, "Tanggal Register")
    lembaga_peradilan = get_detail(table, "Lembaga Peradilan")
    jenis_lembaga_peradilan = get_detail(table, "jenis Lembaga Peradilan")
    hakim_ketua = get_detail(table, "Hakim Ketua")
    hakim_anggota = get_detail(table, "Hakim Anggota")
    panitera = get_detail(table, "Panitera")
    amar = get_detail(table, "Amar")
    amar_lainnya = get_detail(table, "Amar Lainnya")
    catatan_amar = get_detail(table, "Catatan Amar")
    tanggal_musyawarah = get_detail(table, "Tanggal Musyawarah")
    tanggal_dibacakan = get_detail(table, "Tanggal Dibacakan")
    kaidah = get_detail(table, "Kaidah")
    abstrak = get_detail(table, "Abstrak")

    try:
        link_pdf = soup.find("a", href=re.compile(r"/pdf/"))["href"]
        file_pdf, file_nama_pdf = get_pdf(link_pdf, path_pdf)
        text_pdf = high_level.extract_text(file_pdf)
        text_pdfc = clean_text(text_pdf)
    except:
        link_pdf = ""
        file_pdf = ""
        file_name_pdf = ""

    data = (
        judul,
        nomor,
        tingkat_proses,
        klasifikasi,
        kata_kunci,
        tahun,
        tanggal_register,
        lembaga_peradilan,
        jenis_lembaga_peradilan,
        hakim_ketua,
        hakim_anggota,
        panitera,
        amar,
        amar_lainnya,
        catatan_amar,
        tanggal_musyawarah,
        tanggal_dibacakan,
        kaidah,
        abstrak,
        link,
        link_pdf,
        file_name_pdf,
        # text_pdf,
    )
    keyword_url = keywoard_url.replace("/", "")
    if keyword_url.startswith("https"):
        keyword_url = "https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika%20dan%20Psikotropika&jenis_doc=putusan&cat=&jd=&tp=&court=098629PN185+++++++++++++++++++++&t_put=&t_reg=&t_upl=2015&t_pr="

    destination = f"{path_output}/_{keyword_url}_{today}"
    print(destination)
    if not os.path.isfile(f"{destination}.text"):
        result.to_csv(f"{destination}.txt", header=True, index=False)
    else:
        result.to_csv(f"{destination}.txt", mode="a", header=False, index=False)

# Sekarang kita buat fungsi utama agar bisa dijalankan di notebook
def run_scraper(keywoard=None, url=None, sort_date=True, download_pdf=True):
    if not keywoard and not url:
        print("Please provide a keywoard or URL")
        return

    path_output = create_path("putusan")
    path_pdf = create_path("pdf-putusan")

    today = date.today().strftime("%Y-%m-%d")

    link = f"https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika%20dan%20Psikotropika&jenis_doc=putusan&cat=&jd=&tp=&court=098629PN185+++++++++++++++++++++&t_put=&t_reg=&t_upl=2015&t_pr="

    if url:
        link = url
    soup = open_page(link)
    last_page = int(
        soup.find_all("a", {"class": "page-link"})[-1].get("data-ci-pagination-page")
    )
    if url:
        print(f"Scraping with url: {url} - {20 * last_page} data - {last_page} page")
    else:
        print(
            f"Scraping with keywoard: {keywoard} - {20 * last_page} data - {last_page} page"
        )
    if url:
        keywoard_url = url
    else:
        keywoard_url = keywoard

    futures = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        for page in range(last_page):
            futures.append(
                executor.submit(run_process, keywoard_url, page + 1, sort_date)
            )
        wait(futures)

def run_process(keywoard_url, page, sort_page):
    if keywoard_url.startswith("https"):
        link = f"{keywoard_url}&page={page}"
    else:
        link = f"https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika%20dan%20Psikotropika&jenis_doc=putusan&cat=&jd=&tp=&court=098629PN185+++++++++++++++++++++&t_put=&t_reg=&t_upl=2015&t_pr={keywoard_url}&page={page}"
    if sort_page:
        link = f"{link}obf=TANGGAL_PUTUSA&obm=desc"

    print(link)

    soup = open_page(link)
    links = soup.find_all("a", {"href": re.compile("/direktori/putusan")})

    for link in links:
        extract_data(link["href"], keywoard_url)

def scrape_specific_url(url, download_pdf=True):
    if not url or not url.startswith("https://"):
        print("please provide a valid URL")
        return

    extract_data(url, url)

In [15]:
run_scraper(url="https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=")

Scraping with url: https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr= - 380 data - 19 page
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=&page=1obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=&page=2obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=&page=3obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=&page=4obf=TANGGAL_PUTUSA&obm=desc
/content/txt_p

In [13]:
run_scraper(keywoard="Narkotika dan Psikotropika", url="https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=")

Scraping with url: https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr= - 380 data - 19 page
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=&page=1obf=TANGGAL_PUTUSA&obm=deschttps://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=&page=2obf=TANGGAL_PUTUSA&obm=desc

https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=&page=3obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2023&t_reg=&t_upl=&t_pr=&page=4obf=TANGGAL_PUTUSA&obm=desc
/content/txt_p

In [14]:
run_scraper(keywoard="Narkotika dan Psikotropika", url="https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86|097488PN86%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&t_put=2022&t_reg=&t_upl=&t_pr=")

Scraping with url: https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86|097488PN86%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&t_put=2022&t_reg=&t_upl=&t_pr= - 380 data - 19 page
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86|097488PN86%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&t_put=2022&t_reg=&t_upl=&t_pr=&page=1obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86|097488PN86%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&t_put=2022&t_reg=&t_upl=&t_pr=&page=2obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=Narkotika&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86|0974

In [16]:
run_scraper(keywoard="Narkotika dan Psikotropika", url="https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2021&t_reg=&t_upl=&t_pr=")

Scraping with url: https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2021&t_reg=&t_upl=&t_pr= - 480 data - 24 page
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2021&t_reg=&t_upl=&t_pr=&page=1obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2021&t_reg=&t_upl=&t_pr=&page=2obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097488PN86++++++++++++++++++++++&t_put=2021&t_reg=&t_upl=&t_pr=&page=3obf=TANGGAL_PUTUSA&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&cou